<a href="https://colab.research.google.com/github/33siwon/AI-networking/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib as plt
import pandas as pd
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms

In [ ]:
path_a1 ='/content/gdrive/MyDrive/example/project2/dataset2/AIcomm_project_2_train_alice_2.csv'
path_p1 ='/content/gdrive/MyDrive/example/project2/dataset2/AIcomm_project_2_train_peter_2.csv'
path_j1 ='/content/gdrive/MyDrive/example/project2/dataset2/AIcomm_project_2_train_john_2.csv'

path_test = '/content/gdrive/MyDrive/example/project2/dataset3/AIcomm_project_2_test (1).csv'

# import csv
##dataset1
alice1 = pd.read_csv(path_a1)
peter1 = pd.read_csv(path_p1)
john1 = pd.read_csv(path_j1)

##testset
test = pd.read_csv(path_test)

# label and nd array
##dataset1
alice1_x = alice1.iloc[:,2:]
alice1_y = alice1.iloc[:,1]

peter1_x = peter1.iloc[:,2:]
peter1_y = peter1.iloc[:,1]

john1_x = john1.iloc[:,2:]
john1_y = john1.iloc[:,1]

test_x = test.iloc[:,2:]
test_y_ = test.iloc[:,1:2]
test_y = test_y_.values.reshape(-1,1)


In [ ]:
test_y_
type(test_y_)

pandas.core.frame.DataFrame

In [ ]:
test_y

array([[3],
       [8],
       [5],
       ...,
       [2],
       [6],
       [0]])

In [ ]:


res_list_0 = [i for i, value in enumerate(test_y) if value == [0]]
res_list_1 = [i for i, value in enumerate(test_y) if value == [1]]
res_list_2 = [i for i, value in enumerate(test_y) if value == [2]]
res_list_3 = [i for i, value in enumerate(test_y) if value == [3]]
res_list_4 = [i for i, value in enumerate(test_y) if value == [4]]
res_list_5 = [i for i, value in enumerate(test_y) if value == [5]]
res_list_6 = [i for i, value in enumerate(test_y) if value == [6]]
res_list_7 = [i for i, value in enumerate(test_y) if value == [7]]
res_list_8 = [i for i, value in enumerate(test_y) if value == [8]]
res_list_9 = [i for i, value in enumerate(test_y) if value == [9]]


print(f"테스트셋 중에서 alice 레이블을 가지는 데이터의 개수",len(res_list_0) + len(res_list_1)+len(res_list_2)+len(res_list_9))
print(f"테스트셋 중에서 peter 레이블을 가지는 데이터의 개수",len(res_list_3) + len(res_list_4)+len(res_list_5)+len(res_list_9))
print(f"테스트셋 중에서 john 레이블을 가지는 데이터의 개수",len(res_list_6) + len(res_list_7)+len(res_list_8)+len(res_list_9))




테스트셋 중에서 alice 레이블을 가지는 데이터의 개수 3943
테스트셋 중에서 peter 레이블을 가지는 데이터의 개수 3936
테스트셋 중에서 john 레이블을 가지는 데이터의 개수 4085


In [ ]:
def sum(label_1,label_2,label_3,label_4):
  sum_ = []
  for i, value in enumerate(label_1):
    a = test_x.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_2):
    a = test_x.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_3):
    a = test_x.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_4):
    a = test_x.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  return final
alice = sum(res_list_0,res_list_1,res_list_2,res_list_9)
len(alice)
alice

In [ ]:
def sum2(label_1,label_2,label_3,label_4):
  sum_ = []
  for i, value in enumerate(label_1):
    a = test_y_.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_2):
    a = test_y_.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_3):
    a = test_y_.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  for i, value in enumerate(label_4):
    a = test_y_.iloc[value:value+1,:]
    sum_.append(a)
  final = pd.concat(sum_)
  return final
alice_y=sum2(res_list_0,res_list_1,res_list_2,res_list_9)
len(alice_y)
alice_y

,label
3,0
12,0
14,0
25,0
30,0
...,...
9962,9
9965,9
9972,9
9973,9


In [ ]:
transform = transforms.Compose(
    [
     transforms.ToPILImage(),
     transforms.ToTensor(),
     transforms.Normalize((0.1307, ), (0.3081, ))
])

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):

        data = self.X.iloc[i, :]
        data = np.asarray(data).astype(np.uint8).reshape(28, 28, 1)
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [ ]:
alice_dataset = MNISTDataset(alice1_x,alice1_y,transform)
peter_dataset = MNISTDataset(peter1_x,peter1_y,transform)
john_dataset = MNISTDataset(john1_x,john1_y,transform)

test_dataset = MNISTDataset(alice ,alice_y  ,transform)



In [ ]:
batch_size = 64
test_batch_size = 1000
epochs = 2
user_id = ['Alice','Peter','john']

In [ ]:
alice_dataloader = DataLoader(alice_dataset, batch_size = batch_size, shuffle = True)
peter_dataloader = DataLoader(peter_dataset, batch_size = batch_size, shuffle = True)
john_dataloader = DataLoader(john_dataset, batch_size = batch_size, shuffle = True)

train_loaders = [alice_dataloader,peter_dataloader,john_dataloader]

test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x.reshape(-1,28*28)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [ ]:
global_model = Net()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

global_model.to(device)
global_weights = global_model.state_dict()

criterion = nn.CrossEntropyLoss()

In [ ]:
def train_test(global_model, train_loaders, user_id, epochs=10, interval=200):
    for epoch in range(epochs):
        local_weights = []
        
        global_model.train()
        
        models = [copy.deepcopy(global_model), copy.deepcopy(global_model), copy.deepcopy(global_model)]
        
        for idx in range(len(user_id)):
            w = local_update(models[idx], train_loaders[idx], user_id[idx], epoch, interval)
            local_weights.append(copy.deepcopy(w))
            
        global_weights = average_weights(local_weights)
        global_model.load_state_dict(global_weights)
    
        test(global_model, test_loader)
    
    return global_model

In [ ]:
def local_update(model, train_loader, user_id, epoch, interval):
    print("User ID : %s" %user_id)
    model.train()
    
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    return model.state_dict()

In [ ]:
def average_weights(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg

In [ ]:
def test(global_model, test_loader):
    global_model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            output = global_model(data)
            test_loss += criterion(output, label).item()
            pred = output.argmax(dim=1, keepdim = True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
global_model = train_test(global_model, train_loaders, user_id)

User ID : Alice
Train Epoch: 0 [0/20000 (0%)]	Loss: 2.226233
Train Epoch: 0 [12800/20000 (64%)]	Loss: 0.037653
User ID : Peter
Train Epoch: 0 [0/20000 (0%)]	Loss: 2.280233
Train Epoch: 0 [12800/20000 (64%)]	Loss: 0.087102
User ID : john
Train Epoch: 0 [0/20000 (0%)]	Loss: 2.410890
Train Epoch: 0 [12800/20000 (64%)]	Loss: 0.118143


KeyError: ignored